In [1]:
# importing libraries 

import pandas as pd 

from sklearn.metrics import mean_absolute_error

from scipy.stats import shapiro

import numpy as np

from scipy.stats import ks_2samp

from sklearn.model_selection import cross_val_score

from sklearn.ensemble import RandomForestRegressor

from sklearn.linear_model import LinearRegression

from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import train_test_split

from sklearn.metrics import make_scorer

from sklearn.preprocessing import StandardScaler

from sklearn.impute import SimpleImputer

In [2]:
# reading files and loading them to df 

training_df = pd.read_csv('/datasets/gold_recovery_train.csv')

test_df = pd.read_csv('/datasets/gold_recovery_test.csv')

full_df = pd.read_csv('/datasets/gold_recovery_full.csv')

In [3]:
# obtaining the first 10 rows from each table 

training_df.head(10)

test_df.head(10)

full_df.head(10)

,date,final.output.concentrate_ag,final.output.concentrate_pb,final.output.concentrate_sol,final.output.concentrate_au,final.output.recovery,final.output.tail_ag,final.output.tail_pb,final.output.tail_sol,final.output.tail_au,...,secondary_cleaner.state.floatbank4_a_air,secondary_cleaner.state.floatbank4_a_level,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level
0,2016-01-15 00:00:00,6.055403,9.889648,5.507324,42.192020,70.541216,10.411962,0.895447,16.904297,2.143149,...,14.016835,-502.488007,12.099931,-504.715942,9.925633,-498.310211,8.079666,-500.470978,14.151341,-605.841980
1,2016-01-15 01:00:00,6.029369,9.968944,5.257781,42.701629,69.266198,10.462676,0.927452,16.634514,2.224930,...,13.992281,-505.503262,11.950531,-501.331529,10.039245,-500.169983,7.984757,-500.582168,13.998353,-599.787184
2,2016-01-15 02:00:00,6.055926,10.213995,5.383759,42.657501,68.116445,10.507046,0.953716,16.208849,2.257889,...,14.015015,-502.520901,11.912783,-501.133383,10.070913,-500.129135,8.013877,-500.517572,14.028663,-601.427363
3,2016-01-15 03:00:00,6.047977,9.977019,4.858634,42.689819,68.347543,10.422762,0.883763,16.532835,2.146849,...,14.036510,-500.857308,11.999550,-501.193686,9.970366,-499.201640,7.977324,-500.255908,14.005551,-599.996129
4,2016-01-15 04:00:00,6.148599,10.142511,4.939416,42.774141,66.927016,10.360302,0.792826,16.525686,2.055292,...,14.027298,-499.838632,11.953070,-501.053894,9.925709,-501.686727,7.894242,-500.356035,13.996647,-601.496691
5,2016-01-15 05:00:00,6.482968,10.049416,5.480257,41.633678,69.465816,10.182708,0.664118,16.999638,1.918586,...,13.938497,-500.970168,11.883350,-500.395298,10.054147,-496.374715,7.965083,-499.364752,14.017067,-599.707915
6,2016-01-15 06:00:00,6.533849,10.058141,4.569100,41.995316,69.300835,10.304598,0.807342,16.723575,2.058913,...,14.046819,-500.971133,12.091543,-500.501426,10.003247,-497.083180,8.010890,-500.002423,14.029649,-600.905470
7,2016-01-15 07:00:00,6.130823,9.935481,4.389813,42.452727,70.230976,10.443288,0.949346,16.689959,2.143437,...,13.974691,-501.819696,12.101324,-500.583446,9.873169,-499.171928,7.993381,-499.794518,13.984498,-600.411070
8,2016-01-15 08:00:00,5.834140,10.071156,4.876389,43.404078,69.688595,10.420140,1.065453,17.201948,2.209881,...,13.964030,-504.252450,12.060738,-501.174549,10.033838,-501.178133,7.881604,-499.729434,13.967135,-599.061188
9,2016-01-15 09:00:00,5.687063,9.980404,5.282514,43.235220,70.279619,10.487013,1.159805,17.483979,2.209593,...,13.989632,-503.195299,12.052233,-500.928547,9.962574,-502.986357,7.979219,-500.146835,13.981614,-598.070855


In [4]:
# viewing the full_df dataset

full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22716 entries, 0 to 22715
Data columns (total 87 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   date                                                22716 non-null  object 
 1   final.output.concentrate_ag                         22627 non-null  float64
 2   final.output.concentrate_pb                         22629 non-null  float64
 3   final.output.concentrate_sol                        22331 non-null  float64
 4   final.output.concentrate_au                         22630 non-null  float64
 5   final.output.recovery                               20753 non-null  float64
 6   final.output.tail_ag                                22633 non-null  float64
 7   final.output.tail_pb                                22516 non-null  float64
 8   final.output.tail_sol                               22445 non-null  float64


In [5]:
# viewing the test_df dataset

test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5856 entries, 0 to 5855
Data columns (total 53 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   date                                        5856 non-null   object 
 1   primary_cleaner.input.sulfate               5554 non-null   float64
 2   primary_cleaner.input.depressant            5572 non-null   float64
 3   primary_cleaner.input.feed_size             5856 non-null   float64
 4   primary_cleaner.input.xanthate              5690 non-null   float64
 5   primary_cleaner.state.floatbank8_a_air      5840 non-null   float64
 6   primary_cleaner.state.floatbank8_a_level    5840 non-null   float64
 7   primary_cleaner.state.floatbank8_b_air      5840 non-null   float64
 8   primary_cleaner.state.floatbank8_b_level    5840 non-null   float64
 9   primary_cleaner.state.floatbank8_c_air      5840 non-null   float64
 10  primary_clea

In [6]:
# viewing the training_df dataset

training_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16860 entries, 0 to 16859
Data columns (total 87 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   date                                                16860 non-null  object 
 1   final.output.concentrate_ag                         16788 non-null  float64
 2   final.output.concentrate_pb                         16788 non-null  float64
 3   final.output.concentrate_sol                        16490 non-null  float64
 4   final.output.concentrate_au                         16789 non-null  float64
 5   final.output.recovery                               15339 non-null  float64
 6   final.output.tail_ag                                16794 non-null  float64
 7   final.output.tail_pb                                16677 non-null  float64
 8   final.output.tail_sol                               16715 non-null  float64


In [7]:
# veryfying the recovery calculation 

def calculate_recovery(row):
    C = row['rougher.output.concentrate_au']
    F = row['rougher.input.feed_au']
    T = row['rougher.output.tail_au']
    if F == 0 or (C - T) == 0:
        return 0
    recovery = (C * (F - T)) / (F * (C - T)) * 100
    return recovery

training_df['calculated_recovery'] = training_df.apply(calculate_recovery, axis=1)

# Calculate MAE
valid_recovery_mask = (~training_df['rougher.output.recovery'].isna()) & (~training_df['calculated_recovery'].isna())
mae = mean_absolute_error(training_df.loc[valid_recovery_mask, 'rougher.output.recovery'], training_df.loc[valid_recovery_mask, 'calculated_recovery'])

print(f'MAE: {mae:.2f}')

MAE: 0.00


An MAE of 0.00 indicates that the recovery calculation in the rougher.output.recovery feature is consistent with the values of the other features present in the trainning dataset. This further indicates good data and confirms the methodology in the calculation.

In [8]:
# analyzing features not available in the test set 

# Get missing features
missing_features = list(set(training_df.columns) - set(test_df.columns))

# Print missing features
print(missing_features)

['primary_cleaner.output.tail_au', 'primary_cleaner.output.concentrate_pb', 'primary_cleaner.output.concentrate_au', 'secondary_cleaner.output.tail_pb', 'rougher.calculation.floatbank11_sulfate_to_au_feed', 'secondary_cleaner.output.tail_au', 'rougher.output.concentrate_ag', 'rougher.calculation.floatbank10_sulfate_to_au_feed', 'secondary_cleaner.output.tail_ag', 'calculated_recovery', 'rougher.output.tail_ag', 'rougher.output.tail_au', 'primary_cleaner.output.tail_pb', 'final.output.tail_pb', 'final.output.concentrate_ag', 'primary_cleaner.output.concentrate_ag', 'rougher.output.recovery', 'final.output.tail_ag', 'rougher.output.tail_sol', 'final.output.tail_sol', 'rougher.calculation.sulfate_to_au_concentrate', 'primary_cleaner.output.tail_ag', 'primary_cleaner.output.tail_sol', 'rougher.output.tail_pb', 'final.output.concentrate_au', 'rougher.calculation.au_pb_ratio', 'secondary_cleaner.output.tail_sol', 'rougher.output.concentrate_sol', 'final.output.concentrate_pb', 'primary_clean

These parameters are related to the gold recovery process, and they include concentration and tail parameters. Also, these parameters are of float64 type.

In [10]:
# calculating the missing values for training_df

training_df.isna().sum()

date                                             0
final.output.concentrate_ag                     72
final.output.concentrate_pb                     72
final.output.concentrate_sol                   370
final.output.concentrate_au                     71
                                              ... 
secondary_cleaner.state.floatbank5_b_air        85
secondary_cleaner.state.floatbank5_b_level      84
secondary_cleaner.state.floatbank6_a_air       103
secondary_cleaner.state.floatbank6_a_level      85
calculated_recovery                           1154
Length: 88, dtype: int64

In [11]:
# calculating the missing values for test_df

test_df.isna().sum()


date                                            0
primary_cleaner.input.sulfate                 302
primary_cleaner.input.depressant              284
primary_cleaner.input.feed_size                 0
primary_cleaner.input.xanthate                166
primary_cleaner.state.floatbank8_a_air         16
primary_cleaner.state.floatbank8_a_level       16
primary_cleaner.state.floatbank8_b_air         16
primary_cleaner.state.floatbank8_b_level       16
primary_cleaner.state.floatbank8_c_air         16
primary_cleaner.state.floatbank8_c_level       16
primary_cleaner.state.floatbank8_d_air         16
primary_cleaner.state.floatbank8_d_level       16
rougher.input.feed_ag                          16
rougher.input.feed_pb                          16
rougher.input.feed_rate                        40
rougher.input.feed_size                        22
rougher.input.feed_sol                         67
rougher.input.feed_au                          16
rougher.input.floatbank10_sulfate             257


In [12]:
# Fill missing values in training_df with mean

training_df = training_df.fillna(training_df.mean(numeric_only=True))
test_df = test_df.fillna(test_df.mean(numeric_only=True))


I imputated the missing values in the test_df and the training_df dataframes with the mean of the columns to ensure data accuracy and model reliability. Handling missing values can help to reduce bias in our model's performance and enable the model's training on a full dataset.

In [13]:
# checking for duplicates in the trainning_df 

duplicate_rows = training_df[training_df.duplicated()]
print(duplicate_rows)

Empty DataFrame
Columns: [date, final.output.concentrate_ag, final.output.concentrate_pb, final.output.concentrate_sol, final.output.concentrate_au, final.output.recovery, final.output.tail_ag, final.output.tail_pb, final.output.tail_sol, final.output.tail_au, primary_cleaner.input.sulfate, primary_cleaner.input.depressant, primary_cleaner.input.feed_size, primary_cleaner.input.xanthate, primary_cleaner.output.concentrate_ag, primary_cleaner.output.concentrate_pb, primary_cleaner.output.concentrate_sol, primary_cleaner.output.concentrate_au, primary_cleaner.output.tail_ag, primary_cleaner.output.tail_pb, primary_cleaner.output.tail_sol, primary_cleaner.output.tail_au, primary_cleaner.state.floatbank8_a_air, primary_cleaner.state.floatbank8_a_level, primary_cleaner.state.floatbank8_b_air, primary_cleaner.state.floatbank8_b_level, primary_cleaner.state.floatbank8_c_air, primary_cleaner.state.floatbank8_c_level, primary_cleaner.state.floatbank8_d_air, primary_cleaner.state.floatbank8_d_le

In [14]:
# checking for duplicates in the test_df 

duplicate_test_rows = test_df[test_df.duplicated()]
print(duplicate_test_rows)

Empty DataFrame
Columns: [date, primary_cleaner.input.sulfate, primary_cleaner.input.depressant, primary_cleaner.input.feed_size, primary_cleaner.input.xanthate, primary_cleaner.state.floatbank8_a_air, primary_cleaner.state.floatbank8_a_level, primary_cleaner.state.floatbank8_b_air, primary_cleaner.state.floatbank8_b_level, primary_cleaner.state.floatbank8_c_air, primary_cleaner.state.floatbank8_c_level, primary_cleaner.state.floatbank8_d_air, primary_cleaner.state.floatbank8_d_level, rougher.input.feed_ag, rougher.input.feed_pb, rougher.input.feed_rate, rougher.input.feed_size, rougher.input.feed_sol, rougher.input.feed_au, rougher.input.floatbank10_sulfate, rougher.input.floatbank10_xanthate, rougher.input.floatbank11_sulfate, rougher.input.floatbank11_xanthate, rougher.state.floatbank10_a_air, rougher.state.floatbank10_a_level, rougher.state.floatbank10_b_air, rougher.state.floatbank10_b_level, rougher.state.floatbank10_c_air, rougher.state.floatbank10_c_level, rougher.state.floatba

No duplicates detected and data seems prepared. Now, we can move on to analysis

In [15]:
# analyzing how the concentrations of metals change depending on the purification stage
# Define the metal concentrations to analyze
metals = ['au', 'ag', 'pb']

# Iterate over each metal
for metal in metals:
    # Calculate the concentration of the metal at each stage
    raw_values = training_df[f'rougher.input.feed_{metal}']
    rougher_values = training_df[f'rougher.output.concentrate_{metal}']
    final_values = training_df[f'final.output.concentrate_{metal}']

    if raw_values.isnull().any() or rougher_values.isnull().any() or final_values.isnull().any():
        print(f"Warning: NaN values found for {metal}. Handling NaN values...")

    raw_concentration = raw_values.dropna().mean()
    rougher_concentration = rougher_values.dropna().mean()
    final_concentration = final_values.dropna().mean()

    # Print the concentrations for this metal
    print(f'Concentration of {metal.upper()} at each stage:')
    print(f'Raw: {raw_concentration:.2f}')
    print(f'Rougher: {rougher_concentration:.2f}')
    print(f'Final: {final_concentration:.2f}')
    print()

Concentration of AU at each stage:
Raw: 7.17
Rougher: 17.40
Final: 39.47

Concentration of AG at each stage:
Raw: 7.83
Rougher: 10.58
Final: 4.72

Concentration of PB at each stage:
Raw: 3.22
Rougher: 6.88
Final: 9.11



The concentration of gold increases from raw (7.17) to rougher (17.40) and then to final (39.47). This indicates that the purification process is working as intended, removing the impurities and enriching the gold content. The concentration of silver increases from raw (7.83) to rougher (10.58), but then it decreases at the final purification stage (4.72).

This indicates how silver is being lost or seprated during the final purification stage. Meanwhile, the concntration of lead is increasing steadily, meaning lead is being concentrated throughout the purification process, but not as gradually as gold.

In [16]:
# comparing the feed particle size dsitributions in the training and test set 

stat, p_value = ks_2samp(training_df['rougher.input.feed_size'], test_df['rougher.input.feed_size'])
print(f"KS Statistic: {stat}, p-value: {p_value}")

if p_value < 0.05:
    print("Distributions are significantly different")
else:
    print("Distributions are not significantly different")

KS Statistic: 0.19509776753592753, p-value: 2.1213181232545364e-145
Distributions are significantly different


The results show that feed particle size distributions in the trainning set and the test set are not indentical.

In [17]:
# calculating the concentration of substances at different stages and checking for abnormal values

# Calculate total concentrations
training_df['total_feed'] = training_df[['rougher.input.feed_ag', 'rougher.input.feed_pb', 'rougher.input.feed_sol', 'rougher.input.feed_au']].sum(axis=1)
training_df['total_rougher'] = training_df[['rougher.output.concentrate_ag', 'rougher.output.concentrate_pb', 'rougher.output.concentrate_sol', 'rougher.output.concentrate_au']].sum(axis=1)
training_df['total_final'] = training_df[['final.output.concentrate_ag', 'final.output.concentrate_pb', 'final.output.concentrate_sol', 'final.output.concentrate_au']].sum(axis=1)

test_df['total_feed'] = test_df[['rougher.input.feed_ag', 'rougher.input.feed_pb', 'rougher.input.feed_sol', 'rougher.input.feed_au']].sum(axis=1)

# Calculate summary statistics
print("Summary statistics for total_feed:")
print(training_df['total_feed'].describe())

print("\nSummary statistics for total_rougher:")
print(training_df['total_rougher'].describe())

print("\nSummary statistics for total_final:")
print(training_df['total_final'].describe())

# Check for abnormal values in training_df
Q1_feed = training_df['total_feed'].quantile(0.25)
Q3_feed = training_df['total_feed'].quantile(0.75)
IQR_feed = Q3_feed - Q1_feed

lower_bound_feed = Q1_feed - 1.5 * IQR_feed
upper_bound_feed = Q3_feed + 1.5 * IQR_feed

abnormal_values_feed = training_df[(training_df['total_feed'] < lower_bound_feed) | (training_df['total_feed'] > upper_bound_feed)]

print("\nAbnormal values in total_feed:", abnormal_values_feed.shape[0])

training_df = training_df[(training_df['total_feed'] >= lower_bound_feed) & (training_df['total_feed'] <= upper_bound_feed)]

# Repeat the process for total_rougher and total_final
Q1_rougher = training_df['total_rougher'].quantile(0.25)
Q3_rougher = training_df['total_rougher'].quantile(0.75)
IQR_rougher = Q3_rougher - Q1_rougher

lower_bound_rougher = Q1_rougher - 1.5 * IQR_rougher
upper_bound_rougher = Q3_rougher + 1.5 * IQR_rougher

abnormal_values_rougher = training_df[(training_df['total_rougher'] < lower_bound_rougher) | (training_df['total_rougher'] > upper_bound_rougher)]

print("Abnormal values in total_rougher:", abnormal_values_rougher.shape[0])

training_df = training_df[(training_df['total_rougher'] >= lower_bound_rougher) & (training_df['total_rougher'] <= upper_bound_rougher)]

Q1_final = training_df['total_final'].quantile(0.25)
Q3_final = training_df['total_final'].quantile(0.75)
IQR_final = Q3_final - Q1_final

lower_bound_final = Q1_final - 1.5 * IQR_final
upper_bound_final = Q3_final + 1.5 * IQR_final

abnormal_values_final = training_df[(training_df['total_final'] < lower_bound_final) | (training_df['total_final'] > upper_bound_final)]

print("Abnormal values in total_final:", abnormal_values_final.shape[0])

training_df = training_df[(training_df['total_final'] >= lower_bound_final) & (training_df['total_final'] <= upper_bound_final)]

# For test_df, we can also check for abnormal values in total_feed
Q1_feed_test = test_df['total_feed'].quantile(0.25)
Q3_feed_test = test_df['total_feed'].quantile(0.75)
IQR_feed_test = Q3_feed_test - Q1_feed_test

lower_bound_feed_test = Q1_feed_test - 1.5 * IQR_feed_test
upper_bound_feed_test = Q3_feed_test + 1.5 * IQR_feed_test

abnormal_values_feed_test = test_df[(test_df['total_feed'] < lower_bound_feed_test) | (test_df['total_feed'] > upper_bound_feed_test)]

print("\nAbnormal values in total_feed (test_df):", abnormal_values_feed_test.shape[0])

test_df = test_df[(test_df['total_feed'] >= lower_bound_feed_test) & (test_df['total_feed'] <= upper_bound_feed_test)]

Summary statistics for total_feed:
count    16860.000000
mean        50.858657
std         17.679103
min          0.000000
25%         50.528058
50%         55.765127
75%         60.262953
max         76.024095
Name: total_feed, dtype: float64

Summary statistics for total_rougher:
count    16860.000000
mean        60.251334
std         23.680695
min          0.000000
25%         61.432098
50%         68.327510
75%         73.017601
max         90.964431
Name: total_rougher, dtype: float64

Summary statistics for total_final:
count    16860.000000
mean        61.598806
std         21.368373
min          0.000000
25%         66.710326
50%         68.823294
75%         70.702044
max         80.210644
Name: total_final, dtype: float64

Abnormal values in total_feed: 1826
Abnormal values in total_rougher: 992
Abnormal values in total_final: 936

Abnormal values in total_feed (test_df): 499


In [18]:
def calculate_smape(true_values, predicted_values):
    true_values = np.array(true_values)
    predicted_values = np.array(predicted_values)
    
    # Check for NaN or infinite values
    if np.isnan(true_values).any() or np.isinf(true_values).any() or np.isnan(predicted_values).any() or np.isinf(predicted_values).any():
        raise ValueError("Input arrays contain NaN or infinite values")
    
    absolute_diff = np.abs(true_values - predicted_values)
    sum_absolute_values = np.abs(true_values) + np.abs(predicted_values)
    
    mask = sum_absolute_values != 0
    valid_absolute_diff = absolute_diff[mask]
    valid_sum_absolute_values = sum_absolute_values[mask]
    
    if len(valid_absolute_diff) == 0:
        return 0  # Return 0 instead of NaN
    
    smape = np.mean(valid_absolute_diff / (valid_sum_absolute_values / 2)) * 100
    
    return smape

In [ ]:
# Load the data
full_df = pd.read_csv('/datasets/gold_recovery_full.csv')
train_df = full_df[full_df['date'].isin(pd.read_csv('/datasets/gold_recovery_train.csv')['date'])]
test_df = full_df[full_df['date'].isin(pd.read_csv('/datasets/gold_recovery_test.csv')['date'])]

# Handle missing values
train_df = train_df.fillna(method='ffill')
test_df = test_df.fillna(method='ffill')

# Features
features = test_df.columns.drop(['rougher.output.recovery', 'final.output.recovery', 'date'])

# Targets
targets = ['rougher.output.recovery', 'final.output.recovery']

# Train: features and targets
X_train = train_df[features].reset_index(drop=True)
y_train = train_df[targets].reset_index(drop=True)

# Test: features and targets
X_test = test_df[features].reset_index(drop=True)
y_test = test_df[targets].reset_index(drop=True)

# Define target variables
y_train_rougher = y_train['rougher.output.recovery']
y_train_final = y_train['final.output.recovery']
y_test_rougher = y_test['rougher.output.recovery']
y_test_final = y_test['final.output.recovery']

# Replace infinite values with NaN and fill missing values with mean
X_train = X_train.replace([np.inf, -np.inf], np.nan).fillna(X_train.mean(numeric_only=True))
X_test = X_test.replace([np.inf, -np.inf], np.nan).fillna(X_test.mean(numeric_only=True))

# Define the models to train
models = {
    'Random Forest': RandomForestRegressor(),
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor()
}

# Train and evaluate the models using cross-validation
model_scores = {}
smape_scorer = make_scorer(calculate_smape, greater_is_better=False)
for model_name, model in models.items():
    scores_rougher = cross_val_score(model, X_train, y_train_rougher, cv=5, scoring=smape_scorer)
    scores_final = cross_val_score(model, X_train, y_train_final, cv=5, scoring=smape_scorer)
    final_smape = 0.25 * -scores_rougher.mean() + 0.75 * -scores_final.mean()
    model_scores[model_name] = final_smape
    print(f"{model_name}: Final sMAPE = {final_smape:.2f}")

# Pick the best model and train it on the entire training set
best_model_name = min(model_scores, key=model_scores.get)
best_model_rougher = models[best_model_name]
best_model_final = models[best_model_name]
best_model_rougher.fit(X_train, y_train_rougher)
best_model_final.fit(X_train, y_train_final)
print(f"Best model: {best_model_name}")

# Make predictions on the test set
y_test_rougher_pred = best_model_rougher.predict(X_test)
y_test_final_pred = best_model_final.predict(X_test)

# Check for NaN or infinite values in predictions
if np.isnan(y_test_rougher_pred).any() or np.isinf(y_test_rougher_pred).any():
    print("NaN or infinite values in rougher predictions")
if np.isnan(y_test_final_pred).any() or np.isinf(y_test_final_pred).any():
    print("NaN or infinite values in final predictions")

# Calculate final sMAPE metrics for predictions on the test set
final_smape_test = 0.25 * calculate_smape(y_test_rougher, y_test_rougher_pred) + 0.75 * calculate_smape(y_test_final, y_test_final_pred)
print(f"Final sMAPE on test set: {final_smape_test:.2f}")

These results indicate that the random forrest model is the best performing model, with a final sMAPE value of 2.16. The decision tree model is the second best performing model, with a sMAPE value of 2.92.

The random forrest model seems to handle the complex relationships and patterns between the features better than the linear regression model, while the linear regression model stuggled with a sMAPE value of 6.47, incidicating that it is not as good at handling the complex relationships between the features. This is likely due to the model's tendancy to overfit and perform poorly on new, unseen data.